## Game Altering Plays
Sean Steinle

1. [Introduction](#1)
2. [Trimming the DF](#2)
3. [Return Positive Plays](#3)
4. [Kick Positive Plays](#4)

<a id='1'>

### Introduction

In determining the value of kickoff returns for the returning team, there are plays that shift field position and there are plays that alter the game. In this notebook, we'll take a deep dive into the latter.

The metrics we'll consider in this notebook are muffs (*isMuff*, *retainsMuff*), fumbles (*isFumble*, *retainsFumble*), and touchdowns (*isTouchdown*). We'll also cut down the dataframe of kickoffs to relevant kickoffs.

### Trimming the DF

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk

In [16]:
df = pd.read_csv("kickoff_dataset.csv")
df

Unnamed: 0 kicking_team receiving_team  year  \
0               5          PIT            TEN  2009   
1              15          MIN            CLE  2009   
2              45          NYJ            HOU  2009   
3              47          NYJ            HOU  2009   
4              48          HOU            NYJ  2009   
...           ...          ...            ...   ...   
28310       28310          SEA             SF  2019   
28311       28311           SF            SEA  2019   
28312       28312          SEA             SF  2019   
28313       28313           SF            SEA  2019   
28314       28314          SEA             SF  2019   

                                                    text        kicker  \
0      J.Reed kicks 61 yards from PIT 30 to TEN 9. J....        J.Reed   
1      R.Longwell kicks 66 yards from MIN 30 to CLE 4...    R.Longwell   
2      J.Feely kicks 72 yards from NYJ 30 to HOU -2. ...       J.Feely   
3      J.Feely kicks 71 yards from NYJ 30 to HOU -1. ...       J.Feely   
4      K.Brown kicks 67 yards from HOU 30 to NYJ 3. L...       K.Brown   
...                                                  ...           ...   
28310  J.Myers kicks 65 yards from SEA 35 to end zone...       J.Myers   
28311  M.Wishnowsky kicks 65 yards from SF 35 to end ...  M.Wishnowsky   
28312  J.Myers kicks 65 yards from SEA 35 to end zone...       J.Myers   
28313  M.Wishnowsky kicks 43 yards from SF 35 to SEA ...  M.Wishnowsky   
28314  J.Myers kicks 44 yards from SEA 35 to SF 21. R...       J.Myers   

       isTouchback  isOutOfBounds  isOnside  isFairCatch  ...  isMuff  \
0            False          False     False        False  ...   False   
1            False          False     False        False  ...   False   
2            False          False     False        False  ...   False   
3            False          False     False        False  ...   False   
4            False          False     False        False  ...   False   
...            ...            ...       ...          ...  ...     ...   
28310         True          False     False        False  ...   False   
28311         True          False     False        False  ...   False   
28312         True          False     False        False  ...   False   
28313        False           True     False        False  ...   False   
28314        False          False     False        False  ...    True   

       retainsMuff  isPenalty penalizedPlayer penaltyYards  \
0             True       True        G.McRath           10   
1             True       True        K.Maiava           10   
2             True       True      C.Thompson           10   
3             True       True      C.Thompson           10   
4             True       True     J.Westerman           10   
...            ...        ...             ...          ...   
28310         True      False       no player            0   
28311         True      False       no player            0   
28312         True      False       no player            0   
28313         True      False       no player            0   
28314         True      False       no player            0   

                 penaltyType  penaltySpot  isFumble  retainsFumble  finalSpot  
0          Offensive Holding           72     False           True         82  
1          Offensive Holding           71     False           True         81  
2          Offensive Holding           70     False           True         80  
3      Unnecessary Roughness           80     False           True         90  
4          Offensive Holding           77     False           True         87  
...                      ...          ...       ...            ...        ...  
28310             no penalty            0     False           True        100  
28311             no penalty            0     False           True        100  
28312             no penalty            0     False           True        100  
28313             no penalty       

In [3]:
#isTouchback, isOutOfBounds, isOnside, isFairCatch, isReturned, isTouchdown, isMuff, retainsMuff, isPenalty, isFumble, retainsFumble
touchback = len(df[df["isTouchback"]])/len(df)
print(f"{touchback*100}% of kicks were touchbacks.")

oob = len(df[df["isOutOfBounds"]])/len(df)
print(f"{oob*100}% of kicks were out of bounds.")

fc = len(df[df["isFairCatch"]])/len(df)
print(f"{fc*100}% of kicks were fair catches.")

returned = len(df[df["isReturned"]])/len(df)
print(f"{returned*100}% of kicks were returned.")

td = len(df[df["isTouchdown"]])/len(df)
print(f"{td*100}% of kicks were returned for a touchdown.")

muff = len(df[df["isMuff"]])/len(df)
print(f"{muff*100}% of kicks were muffed.")

cut = df[df[""]]
rm = len(cut[cut["retainsMuff"]])/len(cut)
print(f"{rm*100}% of isMuffmuffs were retained.")

penalty = len(df[df["isPenalty"]])/len(df)
print(f"{penalty*100}% of kickoffs had an accepted penalty.")

fumble = len(df[df["isFumble"]])/len(df)
print(f"{fumble*100}% of kicks were fumbled.")

cut = df[df["isFumble"]]
rf = len(cut[cut["retainsFumble"]])/len(cut)
print(f"{rf*100}% of fumbles were retained.")

#These numbers look to be about right. One immediate comparison we can do is fumbles lost and muffs kicks vs. touchdowns.

46.40649832244394% of kicks were touchbacks.
1.161928306551298% of kicks were out of bounds.
0.1553946671375596% of kicks were fair catches.
51.66166342927777% of kicks were returned.
0.42733533462828893% of kicks were returned for a touchdown.
0.8935193360409677% of kicks were muffed.
90.11857707509881% of muffs were retained.
4.043793042556949% of kickoffs had an accepted penalty.
1.0665724880805227% of kicks were fumbled.
46.35761589403973% of fumbles were retained.


In [18]:
df = df[df["isReturned"]]
len(df)

14594

In [40]:
muff = len(df[df["isMuff"]])/len(df)
print(f"{muff*100}% of kicks were muffed.")
cut = df[df["isMuff"]]
rm = len(cut[cut["retainsMuff"]])/len(cut)
print(f"{rm*100}% of muffs were retained.")
muff_loss = (muff*100)*(1-rm)
print(f"{muff_loss}% of returned kicks were muffed and lost.")

fumble = len(df[df["isFumble"]])/len(df)
print(f"{fumble*100}% of kicks were fumbled.")
cut = df[df["isFumble"]]
rf = len(cut[cut["retainsFumble"]])/len(cut)
print(f"{rf*100}% of fumbles were retained.")
fumble_loss = (fumble*100)*(1-rf)
print(f"{fumble_loss}% of returned kicks were fumbled and lost")

print(f"{muff_loss + fumble_loss}% of returned kicks were lost due to fumble or a muffed catch.")

td = len(df[df["isTouchdown"]])/len(df)
print(f"{td*100}% of kicks were returned for a touchdown.") #WHO SCORED THE TOUCHDOWN?
#you need to figure out how
list(df[df["isTouchdown"]]["text"])

1.4526517747019323% of kicks were muffed.
92.45283018867924% of muffs were retained.
0.1096340962039194% of returned kicks were muffed and lost.
1.8432232424283952% of kicks were fumbled.
40.89219330855018% of fumbles were retained.
1.0894888310264492% of returned kicks were fumbled and lost
1.1991229272303685% of returned kicks were lost due to fumble or a muffed catch.
0.8222557215293956% of kicks were returned for a touchdown.


['J.Nedney kicks 71 yards from SF 30 to MIN -1. P.Harvin for 101 yards, TOUCHDOWN.',
 'J.Hanson kicks 72 yards from DET 30 to CHI -2. J.Knox for 102 yards, TOUCHDOWN.',
 'S.Lechler kicks 75 yards from OAK 20 to HOU 5. J.Jones for 95 yards, TOUCHDOWN.',
 'J.Reed kicks 68 yards from PIT 30 to CLE 2. J.Cribbs for 98 yards, TOUCHDOWN.',
 'R.Lloyd kicks 67 yards from CAR 30 to TB 3. S.Stroughter for 97 yards, TOUCHDOWN.',
 'N.Kaeding kicks 63 yards from SD 30 to DEN 7. E.Royal for 93 yards, TOUCHDOWN.',
 'J.Reed kicks 58 yards from PIT 30 to MIN 12. P.Harvin for 88 yards, TOUCHDOWN. Back judge Rich Reels injured on the play.',
 'M.Prater kicks 65 yards from DEN 30 to BAL 5. L.Webb for 95 yards, TOUCHDOWN.',
 'J.Feely kicks 70 yards from NYJ 30 to MIA 0. T.Ginn for 100 yards, TOUCHDOWN.',
 'J.Feely kicks 71 yards from NYJ 30 to MIA -1. T.Ginn for 101 yards, TOUCHDOWN.',
 'Josh.Brown kicks 67 yards from STL 30 to NO 3. C.Roby for 97 yards, TOUCHDOWN.',
 'J.Reed kicks 66 yards from PIT 30 to C